In [ ]:
# put notebook in PGPR folder with main code to execute cells below
# train transE embeddings using `train_transe_model.py` script

In [12]:
from utils import *
from train_transe_model import *

In [2]:

class Object(object):
    pass

args = Object()
args.log_dir = './tmp_embed/mooc/train_transe_model'
args.epochs = 10
args.use_user_relations = False

kg_args = Object()
kg_args.entities = ['user', 'item']
kg_args.user_relation = {}
kg_args.item_relation = {
            
        }
kg_args.interaction = 'enroll'

args.use_wandb = False
args.tmp_dir = './transe_embeds'

extract_embeddings(args, kg_args)

Load embeddings ./tmp_embed/mooc/train_transe_model/transe_model_sd_epoch_10.ckpt


In [3]:
embeds = load_embed('./transe_embeds')

Load embedding: ./transe_embeds/transe_embed.pkl


In [4]:
num_users = embeds['user'].shape[0]
num_items = embeds['item'].shape[0]
embeds['user'].shape, embeds['item'].shape

((6548, 100), (687, 100))

In [5]:
embeds_enroll_reshaped = embeds['enroll'][0].reshape((-1, 100))
embeds_enroll_reshaped.shape

(1, 100)

In [6]:
hr = (embeds['user'] + embeds_enroll_reshaped)
tail = embeds['item']

In [7]:
scores = np.dot(hr, tail.T)
scores.shape

(6548, 687)

In [8]:
train_records = {u: set() for u in range(num_users)}
test_records = {u: set() for u in range(num_users)}

with open('./data/mooc/files/train.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        user_course = line.split('\t')

        user = user_course[0]
        course = user_course[1]

        train_records[int(user)].add(int(course))


with open('./data/mooc/files/test.txt', 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    for line in lines:
        user_course = line.split('\t')

        user = user_course[0]
        course = user_course[1]

        test_records[int(user)].add(int(course))

In [20]:
# Recommender - evaluation:

item_set = set(range(num_items))
user_set = set(range(num_users))

k = 10

user_embeds = torch.tensor(hr)
item_embeds = torch.tensor(tail)

precisions, recalls, ndcgs, hit = [], [], [], []
for user in user_set:
    test_item_list = list(item_set - train_records[user])
    item_score_map = {}
    scores = torch.nn.functional.cosine_similarity(item_embeds[test_item_list], user_embeds[[user]])

    for i, s in zip(test_item_list, scores):
        item_score_map[i] = s.item()

    item_score_pair_sorted = sorted(item_score_map.items(), key=lambda x: x[1], reverse=True)
    item_sorted = [i[0] for i in item_score_pair_sorted]

    dcg = 0.0
    for i in item_sorted[:k]:
        if i in test_records[user]:
            dcg += 1. / (np.log(i + 2) / np.log(2))
    idcg = 0.0
    for i in range(min(len(item_sorted[:k]), len(test_records[user]))):
        idcg += 1. / (np.log(i + 2) / np.log(2))
    ndcg = dcg / idcg

    hit_num = len(set(item_sorted[:k]) & test_records[user])
    precisions.append(hit_num / k)
    recalls.append(hit_num / len(test_records[user]))
    ndcgs.append(ndcg)

    print(set(item_sorted[:k]),test_records[user], hit_num)

print(f'precision: {np.mean(precisions)}')
print(f'recall: {np.mean(recalls)}')
print(f'ndcg: {np.mean(ndcgs)}')


{6, 70, 202, 108, 143, 315, 85, 22, 27, 157} {8, 6} 1
{321, 133, 199, 11, 141, 239, 177, 20, 84, 152} {19, 11, 20} 2
{64, 33, 290, 71, 76, 79, 47, 22, 188, 29} {33, 29, 22} 3
{199, 39, 46, 239, 144, 15, 243, 283, 60, 127} {46, 14, 39} 2
{287, 4, 41, 234, 239, 15, 85, 152, 157, 255} {4, 13, 47, 54, 59} 1
{8, 79, 48, 82, 52, 22, 23, 121, 251, 255} {64, 52, 69} 1
{32, 3, 199, 79, 214, 23, 217, 27, 30, 63} {27, 30, 23} 3
{64, 65, 100, 71, 8, 235, 78, 82, 23, 27} {66, 27, 78} 2
{65, 35, 37, 6, 104, 206, 14, 81, 90, 27} {37, 81, 89, 90, 27} 4
{416, 35, 100, 45, 81, 86, 89, 346, 127, 95} {100, 45, 86, 95} 4
{65, 3, 6, 71, 105, 106, 79, 82, 370, 317} {65, 105, 82, 106} 4
{357, 37, 199, 39, 110, 177, 243, 116, 117, 95} {114, 116, 117, 110} 3
{36, 136, 123, 108, 79, 48, 243, 56, 27, 95} {56, 122, 123} 2
{32, 70, 91, 235, 12, 147, 22, 27, 60, 95} {70, 95} 2
{100, 136, 137, 239, 177, 81, 84, 407, 152, 60} {136, 137, 133, 142} 2
{66, 199, 391, 204, 55, 150, 23, 56, 347, 287} {56, 145} 1
{128, 164, 